In [110]:
import numpy as np
import copy
from datetime import datetime
import numpy as np
import plotly.figure_factory as ff
import pandas as pd

In [111]:
kapa = 3 #kapazität der Fahrzeuge
M = 1000 
I = [0,1,2,3,4,5] #5 Kunden+Depot = Knoten
K = 2 #Anzahl Fahrzeuge = Anzahl Touren
seed = [] #Vordefinierte Seed Customer
c = np.array([[0,12,24,10,24,25],[12,0,24,21,14,27],
              [24,24,0,32,37,48],[10,21,32,0,30,22],[24,14,37,30,0,25],[25,27,48,22,25,0]]) #Kostenmatrix, hier Distanz in Minuten
startzeitfenster = [0, 20, 10, 50, 55, 70, 0]
endezeitfenster = [100, 40, 35, 70, 65, 75,100]
kundenOffen = copy.deepcopy(I)
kundenOffen.remove(0)

In [112]:
c

array([[ 0, 12, 24, 10, 24, 25],
       [12,  0, 24, 21, 14, 27],
       [24, 24,  0, 32, 37, 48],
       [10, 21, 32,  0, 30, 22],
       [24, 14, 37, 30,  0, 25],
       [25, 27, 48, 22, 25,  0]])

In [113]:
kunden = [i for i in kundenOffen if i not in seed] #Alle Kunden die noch nicht im Tourenplan sind
tourenplaninit = []
tourenplaninit.append([0,0]) #hier manuell vorgegeben, sollte leer sein
tourenplaninit.append([0,0])

In [114]:
tourenplaninit

[[0, 0], [0, 0]]

In [115]:
tourenplan = copy.deepcopy(tourenplaninit)

In [116]:
kunden

[1, 2, 3, 4, 5]

In [117]:
def parinsertion (tourenplan, kunden, M, K, kapa, startzeitfenster, endezeitfenster):
    szf = copy.deepcopy(startzeitfenster)
    ezf = copy.deepcopy(endezeitfenster)
    #in jeder Iteration wird ein Kunde aus der Liste genommen, bis diese leer ist
    while kunden != []:
        pOpt = M #initialisiert mit sehr hoher Zahl M
        for j in kunden:
            
            #K ist die Anzahl der Touren/Fahrzeuge
            for k in range(K): 
               
                #pos ist die Position in Tour k, an der der Kunde eingefügt wird
                for pos in range(1, len(tourenplan[k])): # iteriert von der zweiten bis zur vorletzen Position der Tour
                    
                    #Feasibility Check: Kann der Kunde an pos eingefügt werden ohne das ein Zeitfenster verpasst wird?    
                    e = max(szf[j],szf[tourenplan[k][pos-1]]+c[tourenplan[k][pos-1],j])
                    l = min(ezf[j],ezf[tourenplan[k][pos]]-c[tourenplan[k][pos],j])
                    if e <= l:
                       
                        kosten = c[tourenplan[k][pos-1],j]+c[j,tourenplan[k][pos]]-c[tourenplan[k][pos-1],tourenplan[k][pos]]#-1 ist letzte position 
                        if  kosten < pOpt: #Verwende die "optimale" Lösung
                            jOpt = j
                            kOpt = k
                            posOpt = pos
                            pOpt = kosten
        if pOpt < M:   
            tourenplan[kOpt].insert(posOpt,jOpt) #Setze denn "optimalen" Kunden, an der "optimalen" Posititon im "optimalen" Tourenplan ein
            #for i in range(posOpt-1, 0, -1):
            #    ezf[tourenplan[k][i]] = min(ezf[tourenplan[k][i]], ezf[tourenplan[k][i+1]]-c[tourenplan[k][i],i+1])
            #for i in range(posOpt, len(tourenplan[k])) :
            #    szf[tourenplan[k][i]] = max(szf[tourenplan[k][i]], szf[tourenplan[k][i-1]]-c[tourenplan[k][i],i-1])
            kunden.remove(jOpt) #Entferne den eigesetzten Kunden aus der Liste der noch ausstehenden Kunden
        #else:
            #print("Diese Kunden konnten nicht eingefügt werden: ", kunden)
            #kunden = []
    return tourenplan

In [118]:
opttourenplan = parinsertion(tourenplaninit, kunden, M, K, kapa, startzeitfenster, endezeitfenster)
print(opttourenplan)


[[0, 1, 3, 5, 0], [0, 2, 4, 0]]


In [119]:
gesamtkost = 0
for k in range(K):
    tourkost = 0
    for pos in range(len(opttourenplan[k])-1):
        tourkost = tourkost + c[opttourenplan[k][pos],opttourenplan[k][pos+1]]
    gesamtkost = gesamtkost + tourkost 
print(gesamtkost) 

165


In [120]:
print(tourenplan)

[[0, 0], [0, 0]]


GANTT CHART

In [57]:
def convert_to_datetime(x):
  return datetime.fromtimestamp(31536000+x*24*3600).strftime("%Y-%m-%d")

In [58]:
start = copy.deepcopy(startzeitfenster[:-1])
ende = copy.deepcopy(endezeitfenster[:-1])

In [59]:
for j in range(len(start)):
    start[j] = convert_to_datetime(start[j])
    ende[j] = convert_to_datetime(ende[j])

In [60]:
num_tick_labels = np.linspace(start = 0, stop = 100, num = 21, dtype = int)
date_ticks = [convert_to_datetime(x) for x in num_tick_labels]

In [61]:
df = pd.DataFrame({'Task': I, "Start": start , "Finish": ende})

In [62]:
fig = ff.create_gantt(df)
fig.layout.xaxis.update({
        'tickvals' : date_ticks,
        'ticktext' : num_tick_labels
        })
fig.show()